In [1]:
import numpy as np
import pandas as pd

# Data

In [5]:
data = pd.read_csv('botswana.tsv', delimiter='\t')

In [6]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

## Task 1

In [7]:
data.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

## Task 2

In [8]:
data_without_missing = data.dropna()

In [9]:
data_without_missing.shape[0]

1834

## Task 3

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
Замените NaN в признаке agefm на c_{agefm}=0c 
agefm
​	
 =0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_{heduc_1}=-1c 
heduc 
1
​	
 
​	
 =−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?



In [23]:
nan = data['agefm'][0]

In [33]:
data['nevermarr'] = data.apply(lambda row: 1 if pd.isna(row['agefm']) else 0, axis=1)

In [35]:
data.drop(columns='evermarr', inplace=True)

In [38]:
data.loc[pd.isna(data['agefm']), 'agefm'] = 0

In [51]:
condition = pd.isna(data['heduc']) & data['nevermarr'] == 1

In [52]:
data.loc[condition, 'heduc'] = -1

In [56]:
pd.isna(data['heduc']).value_counts()

False    4238
True      123
Name: heduc, dtype: int64

## Task 4

In [57]:
data['idlnchld_noans'] = data.apply(lambda row: 1 if pd.isna(row['idlnchld']) else 0, axis=1)
data['heduc_noans'] = data.apply(lambda row: 1 if pd.isna(row['heduc']) else 0, axis=1)
data['usemeth_noans'] = data.apply(lambda row: 1 if pd.isna(row['usemeth']) else 0, axis=1)

In [58]:
data.loc[data['idlnchld_noans'] == 1, 'idlnchld'] = -1
data.loc[data['heduc_noans'] == 1, 'heduc'] = -2
data.loc[data['usemeth_noans'] == 1, 'usemeth'] = -1

In [60]:
data = data.dropna()

In [61]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4357,2,28,7,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4358,4,24,5,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1,0,0,0
4359,1,26,0,spirit,5.0,1.0,0.0,22.0,7.0,0,0.0,1.0,0.0,0.0,0,0,0,0


In [62]:
4348*18

78264

## Task 5

In [63]:
import statsmodels.formula.api as smf

In [72]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + radio + tv + bicycle +'\
             'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
             data=data)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 29 Apr 2021   Prob (F-statistic):               0.00
Time:                        18:56:28   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

## Task 7

In [73]:
import statsmodels.stats.api as sms

In [74]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [80]:
m5 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', data=data)
fitted = m5.fit()

In [81]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [82]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m5.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [83]:
m6 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + '\
                    'agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans', data=data)
fitted = m6.fit()

In [85]:
print(m5.fit().compare_f_test(m6.fit()))

(92.89058230109802, 3.1552009480371243e-40, 2.0)
